In [2]:
!pip install folium

In [3]:
import pandas as pd
import folium

In [4]:
df=pd.read_csv("Police_Department_Incidents_-_Previous_Year__2016_.csv")

In [5]:
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


In [6]:
df.shape

(150500, 13)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150500 entries, 0 to 150499
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IncidntNum  150500 non-null  int64  
 1   Category    150500 non-null  object 
 2   Descript    150500 non-null  object 
 3   DayOfWeek   150500 non-null  object 
 4   Date        150500 non-null  object 
 5   Time        150500 non-null  object 
 6   PdDistrict  150499 non-null  object 
 7   Resolution  150500 non-null  object 
 8   Address     150500 non-null  object 
 9   X           150500 non-null  float64
 10  Y           150500 non-null  float64
 11  Location    150500 non-null  object 
 12  PdId        150500 non-null  int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 14.9+ MB


In [8]:
df1 = df.iloc[0:150, :]

In [9]:
df1.shape

(150, 13)

In [10]:
# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42

### Creating a map of san francisco by adding its latitude and longitude.

In [11]:
sanf_map=folium.Map(location=[latitude,longitude], zoom_start=12)
sanf_map

### Taking 150 crimes from the dataset and using their loctaion to show where the crimes happened 

In [12]:
incidents = folium.map.FeatureGroup()

# loop through the 150 crimes and add each to the incidents feature group
for lat, lng, in zip(df1.Y, df1.X):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='pink',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanf_map.add_child(incidents)

### Adding markers and when you click them it shows the type of crime that happened there.

In [14]:
incidents = folium.map.FeatureGroup()

for lat, lng, in zip(df1.Y, df1.X):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(df1.Y)
longitudes = list(df1.X)
labels = list(df1.Category)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(sanf_map)    
    
sanf_map.add_child(incidents)

### Removing the markers as they were taking a lot space but keeping the labels to show the type of crime.

In [16]:
sanf_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df1.Y, df1.X, df1.Category):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sanf_map)

sanf_map

### Grouping the markers into different clusters. Each cluster is then represented by the number of crimes in each neighborhood.

In [17]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sanf_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sanf_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df1.Y, df1.X, df1.Category):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

sanf_map

# The END